# Augment data

In [ ]:
from mxnet import nd, gluon, init, autograd, image
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from mxnet.gluon import data as gdata
from IPython import display
import matplotlib.pyplot as plt
import time
import os
import numpy as np
import csv
import mxnet
from PIL import Image
import pandas as pd

In [ ]:
path = r"C:\Users\Una\hand_resize"
pic_list = os.listdir(path)

In [ ]:
color_aug = gdata.vision.transforms.RandomColorJitter(
    brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)

augs = gdata.vision.transforms.Compose([
    gdata.vision.transforms.RandomFlipLeftRight(), color_aug,
    gdata.vision.transforms.RandomFlipTopBottom()])

In [ ]:
for file in pic_list:
    with open(os.path.join(path,file), 'rb') as fp:
        str_image = fp.read()
    img = image.imdecode(str_image)
    for j in range(3):
        aug_img = augs(img)
        img_save = Image.fromarray(aug_img.asnumpy(), 'RGB')
        img_save.save(os.path.join(r"C:\Users\Una\hand_aug", str(file[:-4])+str(j)+str(file[-4:])),"JPEG"  )

# Resize images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import pandas as pd

In [ ]:
classes = ["class0","class1","class2","class3","class4","class5"]
ori_path = r"C:\Users\Una\data\train"
toward_path = r"SIGNS\train_signs"

for class_i in classes:
    class_path = os.path.join(ori_path,class_i)
    class_pic_list = os.listdir(class_path)
    for i in range(len(class_pic_list)):
        img = Image.open(os.path.join(class_path,class_pic_list[i]))
        img = img.resize((256,256))
        img.save(os.path.join(toward_path,class_i,class_pic_list[i]) , 'JPEG')

# Training process

In [1]:
from mxnet import nd, gluon, init, autograd, image
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from mxnet.gluon import data as gdata
from IPython import display
import matplotlib.pyplot as plt
import time
import os
import numpy as np
import csv
import mxnet
from PIL import Image
import mxnet as mx

In [2]:
import random
import time
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [3]:
def acc(output, label):
    return (output.argmax(axis=1) ==
            label.astype('float32')).mean().asscalar()

In [4]:
batch_size = 16

In [5]:
transformer = transforms.Compose([
    transforms.ToTensor(),
])

train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(r"SIGNS\train_signs",flag=1,
    transform = lambda data, label: (data.astype(np.float32)/255, label)).transform_first(transformer),
    batch_size=batch_size, shuffle=True, num_workers=0)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(r"SIGNS\test_signs",flag=1,
    transform = lambda data, label: (data.astype(np.float32)/255, label)).transform_first(transformer),
    batch_size=batch_size, shuffle=True, num_workers=0)

In [6]:
epochs = 10
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Transfer Learning

In [7]:
from mxnet.gluon.model_zoo.vision import resnet50_v1
pretrained_net = resnet50_v1(pretrained=True)
print(pretrained_net)

ResNetV1(
  (output): Dense(2048 -> 1000, linear)
  (features): HybridSequential(
    (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm(fix_gamma=False, momentum=0.9, use_global_stats=False, eps=1e-05, axis=1, in_channels=64)
    (2): Activation(relu)
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (4): HybridSequential(
      (0): BottleneckV1(
        (downsample): HybridSequential(
          (0): Conv2D(64 -> 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm(fix_gamma=False, momentum=0.9, use_global_stats=False, eps=1e-05, axis=1, in_channels=256)
        )
        (body): HybridSequential(
          (0): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm(fix_gamma=False, momentum=0.9, use_global_stats=False, eps=1e-05, axis=1, in_channels=64)
          (2): Activation(relu)
          (3): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), pa

In [ ]:
net = resnet50_v1(classes=6)
net.features = pretrained_net.features
net.output.initialize(init.Xavier())

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.05})
best_acc = 0
for epoch in range(epochs):
    train_loss, train_acc, valid_acc = 0., 0., 0.
    tic = time.time()
    #for batch_idx, (data, label) in enumerate(data_loader):
    for (data, label) in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output,label)
        loss.backward()

        # update parameters
        trainer.step(batch_size)
        # calculate training metrics
        train_loss += loss.mean().asscalar()
        train_acc += acc(output, label)
        
    if(np.isnan(train_loss)):
        print("Training ends")
        break
    
    # calculate validation accuracy
    for data, label in test_data:
        valid_acc += acc(net(data), label)
    if(valid_acc > best_acc):
        best_acc = valid_acc
        net.save_parameters('net_resnet50_v1.params')  
    print("Epoch %d: loss %.3f, train acc %.3f, test acc %.3f, in %.1f sec" % ( 
            epoch, train_loss/len(train_data), train_acc/len(train_data),
            valid_acc/len(test_data), time.time()-tic))